<a href="https://colab.research.google.com/github/Laurene-v/Poem_generation_through_phonetics/blob/main/Rhyme_phonetic_generationRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Rhyme generation based on phonetic data with an RNN (based on Andrej Karpathy's RNN)




This tutorial demonstrates how to generate text using a character-based RNN. 

## Setup

### Import TensorFlow and other libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

### Download the dataset


In [ ]:
from google.colab import drive

drive.mount("/content/drive")
wdir = "/content/drive/MyDrive/data"
os.chdir(wdir)

Mounted at /content/drive


In [ ]:

#path_to_file = "adele.txt"
#path_to_file = "adele_phonetics.txt"
#path_to_file = "merged_lyrics.txt"
#path_to_file = "merged_lyrics_phonetics.txt"
#path_to_file = "merged_quatrains_cleaned.txt"
path_to_file = "merged_quatrains_cleaned_phonetics.txt"
#path_to_file = "same_two_rhymes_cleaned_phonetics.txt"



text = open(path_to_file,"rb").read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

Length of text: 19886345 characters


In [ ]:
print(text[:1000])

wɪʧˌkɔrəˈspɑndztɪəˈdɪstənˈnɪrˈbaɪ
ɪzəfriwɛbˈhoʊstɪŋˈsərvɪs,pərˈsɪstəntaɪ
ɪzəˌɔdioʊˈvɪʒuəlˈɑrtɪstˌəndərˈlaɪ
ɪzənˈælkəˌhɑləˈbjuzpriˈvɛnʃənsˈmɑrtəsslaɪ

ɪzðəˈdeɪbjuˈstudiˌoʊˈælbəməkɑn
001əv3ˈfɛbruˌɛri1997wɪʧwɑzsaɪndˌðɛˈrəməˈkɑn
003230ɪzəsaʊθˌkɔˈriənfudˌmænjəˈfækʧərərˌpɛrɪgˈnɑn
ˈmɑləˌkjulzɪnsɔɪlərɪbɑn

əvðəvoʊtswərˌɪnˈvælədaɪˈwaɪ
ɪzðəfɪθˈstudiˌoʊˈælbəmbaɪðduˈbaɪ
005ɪzə1981ɑrˈkeɪdgeɪmbaɪˈsigəriˈtraɪ
007ˈlɛʤəndzɪzəˈʃutərsuˈnigdi

007ˈlɛʤəndzɪzðəfɔrθəndˈfaɪnəlʤeɪmˈæmən
007ˈreɪsɪŋɪzəˈreɪsɪŋˈvɪdioʊgeɪmfleɪmkwɑn
00æməndbərndəˈtoʊtələˌbuˈtæn
tɪɔnəndɪtɪzðəˈbɪgəstəˈbəfɔrˈgɑn

ɪzəlɪvˈælbəmbariˈtraɪ
ɪzkənˈsɪdərdtɪbiðəstɑrˈdiˈjuˈaɪ
00piɛmɪnˈdeɪligoʊzɔnˈriˈrənˈproʊˌgræmlaɪ
00ðəˈrɛbəlzˈstɑrtɪdtɪθroʊˈɑbʤɛktsˈkɪləˌgræmflaɪ

ɛzˈnoʊtɪdɪnðəˈkætəlɔgrɑn
010biɪŋðəfərðəˈsɪtkɑn
010ɪzðəˈdeɪbjuˈstudiˌoʊˈælbəmbagəˈbɑn
dɪˈnoʊtstɪˈʤænjuˌɛriˈælbəmkaprɑn

ˌspɛsəfəˈkeɪʃənzˌrikˈwaɪəslaɪ
0114ɪzðəˈnæʃənəlˈdaɪəlɪŋkoʊdfərsəˈplaɪəəˈlaɪ
ˌrɛvəˈluʃənɪnʤɪstˈlɪsənɪfraɪ
0137wɑzəˈʤərməntɔkˈkrɪsənɪdi



In [ ]:

vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))
print(vocab)

70 unique characters
['\n', '\r', '!', '"', '#', '$', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '`', 'a', 'b', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'z', '~', 'æ', 'ð', 'ŋ', 'ɑ', 'ɔ', 'ə', 'ɛ', 'ɪ', 'ʃ', 'ʊ', 'ʒ', 'ʤ', 'ʧ', 'ˈ', 'ˌ', 'θ']


## Trying other RNN
This is the smaller version from Andrej Karpathy's github

In [ ]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open(path_to_file,"rb").read().decode(encoding='utf-8') # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print ('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: 
    print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
ˌæbəlmæˈsɛriz,ɪzmænˈtiʃərˈændɪʤˈʤɪlənəstruˈælənəgrɪˈrɛmbərðəˈlitaʊskɑˈræstiˈridˌənbæθərrækˈbɑkrəlˈbɔrdʤæzˈbɑltɪ,ˈpdunˌɪrˈdiˌjoʊˈˈθɛrmənəlˌɑvɪnəgjuˈlaɪtəlpɔrlˈʤɑn
ˌeɪtjuʃəˈkɑ 
----
iter 77900, loss: 54.226677
----
 ʊ"pɪnmɪˈlaɪnzhɪsboʊzfrəblaɪ

waɪ
ˌoʊbgɑn
ɪzðərˈʤɪnəfənʤɪzəˈbɑrʃaɪˌlɪkljuˈstuˈsunətˈkəndibɪswnɛrˈæbsən,əvˈliʃioʊaɪ

əɪʃəˈwɪbənɪˈtərʧænˈʃuvəˈməvərˈpɪsəlɛktiˈhɑduðəˈnaɪləlɛlstirəvðərəbəvwaɪ
ˌsɛkʃɪ 
----
iter 78000, loss: 53.858870
----
 zˈgoʊʃənaɪ
əˈʤɑn
əˈbɛlʃən,lrmeɪstˈsudəlblaɪkboʊməlmɑn

əˈbɔrsɪgˈækʃənəˈweɪniʃɑn
ɪzi
əbɪˈmɑnʃənəkənəbɔrgɪˈsaɪ
əɪsɪnˈfɔlməgˈlɑn
əbɔnˈbɪʃənʃənˈdiləlˈkliʃɪzˈspjoʊbiəraɪ
ə1bɔrnɑbrɪˈwɑzəpɪzˈbaɪəlˈɪ 
----
iter 78100, loss: 53.613139
----
 ɪˈlæniənɪblɑn
ˈeɪŋɪsɪnˈɔlɪksɪˌɪzfərlˈɪnvɛˌɪnˈʃɑn
əˌɪvɪgˈɪnjənaɪ
ˈbɔɪnzwɑzəˈɑnməlbɪzɑn

əbɔknˌrɛʤɪnzɪgdˈsiˌsuˈpɑzɪˈʃɑˈbeɪnˈzɑn
əˈbɔrkənˈmeɪʃiəloʊfərˈʤinəˈɪtɪlˌˈɔpiˈæɪlərəˈbaɪ
əəbaɪkərsərˈbɔrʃənˌ 
----
iter 78200, loss: 52.878783
----
 eɪvðəˈsp

KeyboardInterrupt: ignored

## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `preprocessing.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars
##########

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `preprocessing.StringLookup` layer:

In [ ]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))

It converts form tokens to character IDs, padding with `0`:

In [ ]:
ids = ids_from_chars(chars)
ids
##########

NameError: ignored

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `preprocessing.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `preprocessing.StringLookup` layer so that the padding and `[UNK]` tokens are set the same way.

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [ ]:
chars = chars_from_ids(ids)
chars
############

NameError: ignored

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()
#############

array([b'ab[UNK]defg', b'[UNK][UNK]z'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(19886345,), dtype=int64, numpy=array([53, 63, 68, ..., 63,  3,  2])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))
################

w
ɪ
ʧ
ˌ
k
ɔ
r
ə
ˈ
s


In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'w' b'\xc9\xaa' b'\xca\xa7' b'\xcb\x8c' b'k' b'\xc9\x94' b'r'
 b'\xc9\x99' b'\xcb\x88' b's' b'p' b'\xc9\x91' b'n' b'd' b'z' b't'
 b'\xc9\xaa' b'\xc9\x99' b'\xcb\x88' b'd' b'\xc9\xaa' b's' b't'
 b'\xc9\x99' b'n' b'\xcb\x88' b'n' b'\xc9\xaa' b'r' b'\xcb\x88' b'b' b'a'
 b'\xc9\xaa' b'\r' b'\n' b'\xc9\xaa' b'z' b'\xc9\x99' b'f' b'r' b'i' b'w'
 b'\xc9\x9b' b'b' b'\xcb\x88' b'h' b'o' b'\xca\x8a' b's' b't' b'\xc9\xaa'
 b'\xc5\x8b' b'\xcb\x88' b's' b'\xc9\x99' b'r' b'v' b'\xc9\xaa' b's' b','
 b'p' b'\xc9\x99' b'r' b'\xcb\x88' b's' b'\xc9\xaa' b's' b't' b'\xc9\x99'
 b'n' b't' b'a' b'\xc9\xaa' b'\r' b'\n' b'\xc9\xaa' b'z' b'\xc9\x99'
 b'\xcb\x8c' b'\xc9\x94' b'd' b'i' b'o' b'\xca\x8a' b'\xcb\x88' b'v'
 b'\xc9\xaa' b'\xca\x92' b'u' b'\xc9\x99' b'l' b'\xcb\x88' b'\xc9\x91'
 b'r' b't' b'\xc9\xaa' b's' b't' b'\xcb\x8c' b'\xc9\x99' b'n'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'w\xc9\xaa\xca\xa7\xcb\x8ck\xc9\x94r\xc9\x99\xcb\x88sp\xc9\x91ndzt\xc9\xaa\xc9\x99\xcb\x88d\xc9\xaast\xc9\x99n\xcb\x88n\xc9\xaar\xcb\x88ba\xc9\xaa\r\n\xc9\xaaz\xc9\x99friw\xc9\x9bb\xcb\x88ho\xca\x8ast\xc9\xaa\xc5\x8b\xcb\x88s\xc9\x99rv\xc9\xaas,p\xc9\x99r\xcb\x88s\xc9\xaast\xc9\x99nta\xc9\xaa\r\n\xc9\xaaz\xc9\x99\xcb\x8c\xc9\x94dio\xca\x8a\xcb\x88v\xc9\xaa\xca\x92u\xc9\x99l\xcb\x88\xc9\x91rt\xc9\xaast\xcb\x8c\xc9\x99n'
b'd\xc9\x99r\xcb\x88la\xc9\xaa\r\n\xc9\xaaz\xc9\x99n\xcb\x88\xc3\xa6lk\xc9\x99\xcb\x8ch\xc9\x91l\xc9\x99\xcb\x88bjuzpri\xcb\x88v\xc9\x9bn\xca\x83\xc9\x99ns\xcb\x88m\xc9\x91rt\xc9\x99ssla\xc9\xaa\r\n\r\n\xc9\xaaz\xc3\xb0\xc9\x99\xcb\x88de\xc9\xaabju\xcb\x88studi\xcb\x8co\xca\x8a\xcb\x88\xc3\xa6lb\xc9\x99m\xc9\x99k\xc9\x91n\r\n001\xc9\x99v3\xcb\x88f\xc9\x9bbru\xcb\x8c\xc9\x9br'
b'i1997w\xc9\xaa\xca\xa7w\xc9\x91zsa\xc9\xaand\xcb\x8c\xc3\xb0\xc9\x9b\xcb\x88r\xc9\x99m\xc9\x99\xcb\x88k\xc9\x91n\r\n003230\xc9\xaaz\xc9\x99sa\xca\x8a\xce\xb8\xcb\x8ck\xc9\x94\xcb\x88ri\xc9\x99nfu

For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())
###########

Input : b'w\xc9\xaa\xca\xa7\xcb\x8ck\xc9\x94r\xc9\x99\xcb\x88sp\xc9\x91ndzt\xc9\xaa\xc9\x99\xcb\x88d\xc9\xaast\xc9\x99n\xcb\x88n\xc9\xaar\xcb\x88ba\xc9\xaa\r\n\xc9\xaaz\xc9\x99friw\xc9\x9bb\xcb\x88ho\xca\x8ast\xc9\xaa\xc5\x8b\xcb\x88s\xc9\x99rv\xc9\xaas,p\xc9\x99r\xcb\x88s\xc9\xaast\xc9\x99nta\xc9\xaa\r\n\xc9\xaaz\xc9\x99\xcb\x8c\xc9\x94dio\xca\x8a\xcb\x88v\xc9\xaa\xca\x92u\xc9\x99l\xcb\x88\xc9\x91rt\xc9\xaast\xcb\x8c\xc9\x99'
Target: b'\xc9\xaa\xca\xa7\xcb\x8ck\xc9\x94r\xc9\x99\xcb\x88sp\xc9\x91ndzt\xc9\xaa\xc9\x99\xcb\x88d\xc9\xaast\xc9\x99n\xcb\x88n\xc9\xaar\xcb\x88ba\xc9\xaa\r\n\xc9\xaaz\xc9\x99friw\xc9\x9bb\xcb\x88ho\xca\x8ast\xc9\xaa\xc5\x8b\xcb\x88s\xc9\x99rv\xc9\xaas,p\xc9\x99r\xcb\x88s\xc9\xaast\xc9\x99nta\xc9\xaa\r\n\xc9\xaaz\xc9\x99\xcb\x8c\xc9\x94dio\xca\x8a\xcb\x88v\xc9\xaa\xca\x92u\xc9\x99l\xcb\x88\xc9\x91rt\xc9\xaast\xcb\x8c\xc9\x99n'


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outpts one logit for each character in the vocabulary. These are the log-liklihood of each character according to the model.

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else: 
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_training.png?raw=1)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details asee the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 72) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [ ]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  18432     
_________________________________________________________________
gru_1 (GRU)                  multiple                  3938304   
_________________________________________________________________
dense_1 (Dense)              multiple                  73800     
Total params: 4,030,536
Trainable params: 4,030,536
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [ ]:
sampled_indices

array([ 0, 54,  8, 38, 68, 36,  7, 12, 17, 64, 34, 11, 47, 29, 33, 54, 39,
       54,  3,  6, 32, 63, 11, 26,  8,  8, 38, 19, 62,  9, 49, 15, 36, 32,
       50, 17, 47, 66, 49, 70, 27, 28, 71, 56, 25, 17, 28, 12, 58,  9, 63,
        6, 69,  3, 27, 48, 13, 60,  4, 28,  5, 24, 50, 16, 37, 11, 58, 40,
       34, 13, 17, 13, 30, 69, 39, 17, 40,  3, 19, 41, 31,  2, 60, 33, 48,
       52, 47, 62, 55, 71, 42, 27, 31, 27,  4, 59, 39,  0, 35, 28])

Decode these to see the text predicted by this untrained model:

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b's\xc9\x99t\xc9\x99n\xcb\x88ka\xca\x8ans\xc9\x99l\xc9\xaa\xcb\x88l\xc9\x9bk\xca\x83\xc9\x99nt\xca\x8akple\xc9\xaad\xc9\x91n\r\n1978s\xcb\x88wid\xc9\xaa\xca\x83gr\xc3\xa6ndpriw\xc9\x91z\xc9\x99\xcb\x88f\xc9\x94rmj\xc9\x99l\xc9\x99\xcb\x8cri\xcb\x88ple\xc9\xaa\xcb\x88ja\xca\x8a\xcb\x88h\xc9\x91n\r\n1978\xcb\x88\xce\xb8\xc9\x99nd\xc9\x99r\xcb\x8cb\xc9\x99rd\xcb\x88'

Next Char Predictions:
 b'z\'g\xca\xa7e$,1\xca\x83b+p@azhz\r#`\xc9\xaa+:\'\'g3\xc9\x9b(s/e`t1p\xca\x92s\xcb\x8c;?\xce\xb8\xc3\xa691?,\xc5\x8b(\xc9\xaa#\xcb\x88\r;r-\xc9\x94!?"8t0f+\xc5\x8bib-1-[\xcb\x88h1i\r3j]\n\xc9\x94arvp\xc9\x9b~\xce\xb8k;];!\xc9\x91hd?'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 72)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.2752547


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [ ]:
tf.exp(mean_loss).numpy() # very close to our vocabulary size (72) so the model seems correctly initialized

71.89845

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

###################################

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
EPOCHS =15

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/15
3076/3076 [==============================] - 172s 55ms/step - loss: 1.1340
Epoch 2/15
3076/3076 [==============================] - 171s 55ms/step - loss: 1.1271
Epoch 3/15
3076/3076 [==============================] - 171s 55ms/step - loss: 1.1224
Epoch 4/15
3076/3076 [==============================] - 171s 55ms/step - loss: 1.1195
Epoch 5/15
3076/3076 [==============================] - 171s 55ms/step - loss: 1.1177
Epoch 6/15
3076/3076 [==============================] - 172s 55ms/step - loss: 1.1164
Epoch 7/15
3076/3076 [==============================] - 171s 55ms/step - loss: 1.1158
Epoch 8/15
3076/3076 [==============================] - 171s 55ms/step - loss: 1.1157
Epoch 9/15
3076/3076 [==============================] - 171s 55ms/step - loss: 1.1163
Epoch 10/15
3076/3076 [==============================] - 171s 55ms/step - loss: 1.1170
Epoch 11/15
3076/3076 [==============================] - 172s 55ms/step - loss: 1.1179
Epoch 12/15
3076/3076 [=============================

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())]) 
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
    predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)
    
    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ɪt'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

print(f"\nRun time: {end - start}")
print("Number of epochs: ", EPOCHS)

ɪtsdɪˈtərməndˈnɛzɪsˌɪntərˈɪtkɑn

ɛzæthəzəˈtaɪtəlgeɪm,"mɔrˈkeɪˈwaɪˈaɪ
ɛzætðwaɪ
ɛzəv2ˌʤuˈlapaɪ
ɛzəv2016mɪstʊrɛnˈhænsən]ˈstudəntʃiˈændərsərɑn
ɛzəv2014ˈoʊvərˈhɛˌdlaɪtəndˌpɑpjəˈleɪʃənˌæfrəˈkeɪʃəˈjɪrˈŋɑn
ɛzəv2014moʊstəvðətaʊn1000ˈstudəntəˈlɑn
ɛzəv2014hihædˈnɛvərbistænmɑˈrɑn
ɛzəv2014hædˈfaʊndɪdfərðəɛˈdiˈdəbəˈljuˈaɪ
ɛzəv2014ɪtwɑzdɪˈvɛləpwɑn
ɛzəv2014ɪthædəˌpɑpjəˈleɪʃənˌənˈkɑmənvaɪ

ɛzəv2014ɪtwɑzðəˈlɑrʤəsthaɪəjoʊˈhɑn
ˈɔlˌstʊdˈɛˌrlaɪnzgrupˈwɪnɪŋzəvðəhɪˈɛˈlɛkˈsaɪ
ɛzəv2007ɪthəzˈmɛnifæksˈjumənˌɪnˈkludɪŋprəˈfɛʃənəlˌwənɔnˈwən

ɛzəv2005ɪtɪzˈprɛzəntəvboʊldɑn
ɛzəv2008ɪtɪzðəˈdeɪbjuˌnɑnˌpəˈlɪkɪbɑn
ɛzəv2008ɪthəzənˈɛsˈsiˈaɪ
ɛzəv2007ɪthəzfɑrswɔn
ɛzəv2005ɪtkənˈtroʊlrɪˈpɔrtədlaɪˈkrɛdɪtɪblaɪ

ɛzəv2008ɪthəzbɪnˈwərkɪŋˈoʊvəˈtaɪˈwɑn
ɛzəv2008ɪthəzənˈɛstəˌmeɪtɪjuɑn
ɛzəv2008ɪtwɑzðəˈfaʊndərəndɪmˈplɔɪiəˈtrɪʃəkraɪ

ɛzəv2008ɪthædəˌpɑpjəˈleɪʃənhəˌwənɔnˈwən
ɛzəv2008ɪtwɑzˈkæmpiənzwənsəˈkjuʃəgɑrˈkaɪ
ɛzəv2008ɪthəz549ˈstudəntˌrɛprɪˈzɛnˈkəˈmɑn
ɛzəv2008ɪthəzəndpoʊəm,ˈkeɪmbrɪʤɪzˌoʊvərˈflaɪ
ɛzəv2007ɪthəz 

__________________________

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result, '\n\n' + '_'*80)


print(f"\nRun time: {end - start}")

tf.Tensor(
[b"ROMEO: disheapellan*; \xca\x83\xc9\x99t ju t\xc9\xaa j\xca\x8ar \xcb\x88f\xc9\x91\xc3\xb0\xc9\x99r ju, \xcb\x8c\xc9\xaan\xcb\x88f\xc9\x9bst\xc9\x99r kra\xc9\xaa di n\xc9\x94r k\xc9\x99n kju \xc3\xb0\xc9\x99 me\xc9\xaa dr\xc9\x94 ni \xc3\xb0\xc9\x99 duk \xc9\x99v d\xc9\x91n! \xcb\x88k\xc3\xa6n\xc9\x91\xca\xa4 t\xc9\xaa \xc3\xb0\xc9\x99 st\xc9\x91k siv \xcb\x88s\xc9\xaal\xc9\xaa\xc5\x8b t\xc9\xaa \xcb\x8cf\xc9\x94rs\xcb\x88we\xc9\xaa. \xca\xa4\xc9\x94\xc9\xaa, \xc3\xb0\xc9\x99t na\xca\x8a \xc9\xaaz bi tlandellity* \xc9\xaan \xcb\x88\xc9\xaa\xc5\x8bgl\xc9\x99nd \xc9\xaan \xc3\xb0\xc9\xaas fr\xc9\x9bndz \xc9\x99v \xc9\x94f; \xc9\x99nd \xcb\x88\xc9\x94r\xc9\x99t\xc9\x99l d\xc9\xaa\xcb\x88za\xc9\xaab\xc9\xaad pre\xc9\xaa \xce\xb8\xc9\x99rk\xcb\x88pla\xca\x8an\xc9\xaad, \xc9\xaa\xcb\x88n\xc9\x99f\xc9\x99s tu, \xc3\xb0\xc9\x99 ka\xc9\xaats \xc9\x99v \xcb\x88h\xc3\xa6pi t\xc9\xaa \xcb\x8c\xc3\xb0\xc9\x9br\xcb\x88f\xc9\x94r ma\xc9\xaa charn'd*. \xcb\x88iz\xc9\x99rd \xc9\xaan mi \xc

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [ ]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO: wik ən ɪn ðə ˈəkˌsɛnt ˈtɛmpər, speɪk ər θru ðaɪ ˈgreɪtəs. ˈʧuliəs: kwin jʊr lɪvz ənd ˈpɪtɪʤ əv ˈsɪg


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.